# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,quatre cocos,-20.21,57.76,73.00,73,75,14.99,MU,1595824907
1,zlitan,32.47,14.57,76.33,77,0,2.71,LY,1595824907
2,lebu,-37.62,-73.65,47.23,83,58,7.58,CL,1595824908
3,lima,-12.04,-77.03,59.00,87,90,9.17,PE,1595824636
4,alyangula,-13.85,136.42,80.60,34,91,12.75,AU,1595824909
...,...,...,...,...,...,...,...,...,...
368,kutum,14.20,24.67,76.75,47,83,4.05,SD,1595825042
369,latung,5.50,120.88,84.16,71,34,6.87,PH,1595825042
370,umm kaddadah,13.60,26.69,79.92,64,46,14.56,SD,1595825042
371,half moon bay,37.46,-122.43,64.40,93,90,4.70,US,1595825043


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store lat and long
locations = weather_data[["Lat", "Lng"]]

#store humidity
humidity = weather_data["Humidity"]

In [5]:
#plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
new_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

new_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,zlitan,32.47,14.57,76.33,77,0,2.71,LY,1595824907
26,lagoa,39.05,-27.98,72.21,93,0,5.61,PT,1595824917
39,carnarvon,-24.87,113.63,71.60,35,0,8.05,AU,1595824921
67,anadyr,64.75,177.48,78.80,54,0,6.71,RU,1595824804
97,ulcinj,41.93,19.22,73.40,56,0,4.70,ME,1595824942
102,bull savanna,17.89,-77.59,76.93,83,0,5.17,JM,1595824943
178,aksehir,38.36,31.42,74.89,48,0,4.41,TR,1595824971
205,alanya,36.54,32.00,71.60,83,0,4.70,TR,1595824789
277,ayia galini,35.10,24.68,79.00,48,0,3.38,GR,1595825006


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = new_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
1,zlitan,LY,32.47,14.57,
26,lagoa,PT,39.05,-27.98,
39,carnarvon,AU,-24.87,113.63,
67,anadyr,RU,64.75,177.48,
97,ulcinj,ME,41.93,19.22,
102,bull savanna,JM,17.89,-77.59,
178,aksehir,TR,38.36,31.42,
205,alanya,TR,36.54,32.00,
277,ayia galini,GR,35.10,24.68,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
# assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={lat},{lng}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city_name} at {lat} , {lng} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    

Retrieving Results for Index 1: zlitan.
Closest hotel to zlitan at 32.47 , 14.57 is مصعب للهاتف النقال.
------------
Retrieving Results for Index 26: lagoa.
Closest hotel to lagoa at 39.05 , -27.98 is QUEIJADAS DA GRACIOSA.
------------
Retrieving Results for Index 39: carnarvon.
Closest hotel to carnarvon at -24.87 , 113.63 is Chicken Treat.
------------
Retrieving Results for Index 67: anadyr.
Closest hotel to anadyr at 64.75 , 177.48 is The Arbitration Court of the Chukotka Autonomous Okrug.
------------
Retrieving Results for Index 97: ulcinj.
Closest hotel to ulcinj at 41.93 , 19.22 is Guest House Lindi & Roni.
------------
Retrieving Results for Index 102: bull savanna.
Closest hotel to bull savanna at 17.89 , -77.59 is JN ATM.
------------
Retrieving Results for Index 178: aksehir.
Closest hotel to aksehir at 38.36 , 31.42 is Akşehir Hidirlik.
------------
Retrieving Results for Index 205: alanya.
Closest hotel to alanya at 36.54 , 32.0 is Hotel Primera.
------------
Retrieving 

In [13]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
1,zlitan,LY,32.47,14.57,مصعب للهاتف النقال
26,lagoa,PT,39.05,-27.98,QUEIJADAS DA GRACIOSA
39,carnarvon,AU,-24.87,113.63,Chicken Treat
67,anadyr,RU,64.75,177.48,The Arbitration Court of the Chukotka Autonomo...
97,ulcinj,ME,41.93,19.22,Guest House Lindi & Roni


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))